In [1]:
import os
import numpy as np
import tensorflow as tf
from magenta.models.melody_rnn import melody_rnn_sequence_generator
from magenta.models.shared import sequence_generator_bundle
from magenta.music import constants
from magenta.music import sequences_lib
from magenta.music import midi_io
from magenta.protobuf import generator_pb2
from magenta.protobuf import music_pb2
from midi2audio import FluidSynth

# Set up environment
os.environ['MAGENTA_GUITAR_BUNDLE_PATH'] = 'lookback_rnn.mag'
BUNDLE_PATH = 'lookback_rnn.mag'  # You'll need to download this model bundle
OUTPUT_DIR = './generated_music'
os.makedirs(OUTPUT_DIR, exist_ok=True)

def generate_melody(primer_melody=None, num_steps=128, temperature=1.0):
    """Generate a melody using the Magenta MelodyRNN model."""
    
    # Initialize the model
    bundle = sequence_generator_bundle.read_bundle_file(BUNDLE_PATH)
    generator_map = melody_rnn_sequence_generator.get_generator_map()
    melody_rnn = generator_map['lookback_rnn'](
        checkpoint=None,
        bundle=bundle)
    melody_rnn.initialize()

    # Create sequence generator options
    generator_options = generator_pb2.GeneratorOptions()
    generator_options.args['temperature'].float_value = temperature
    generator_options.generate_sections.add(
        start_time=0,
        end_time=num_steps * constants.DEFAULT_STEPS_PER_QUARTER / 4.0)

    # Create a sequence
    if primer_melody:
        melody_sequence = primer_melody
    else:
        # Create an empty sequence
        melody_sequence = music_pb2.NoteSequence()
        melody_sequence.tempos.add().qpm = 120
        melody_sequence.ticks_per_quarter = constants.STANDARD_PPQ

    # Generate the sequence
    generated_sequence = melody_rnn.generate(melody_sequence, generator_options)

    return generated_sequence

def save_midi(sequence, output_file):
    """Save the sequence as a MIDI file."""
    sequence_adjusted = sequences_lib.quantize_note_sequence(
        sequence, steps_per_quarter=4)
    midi_io.sequence_proto_to_midi_file(sequence_adjusted, output_file)
    return output_file

def midi_to_audio(midi_file, output_file):
    """Convert MIDI to audio using FluidSynth."""
    fs = FluidSynth()
    fs.midi_to_audio(midi_file, output_file)
    return output_file

def generate_and_save_music(num_sequences=5, temperature=1.0):
    """Generate multiple music sequences and save them as MIDI and WAV files."""
    for i in range(num_sequences):
        # Generate a melody
        print(f"Generating sequence {i+1}/{num_sequences}...")
        sequence = generate_melody(temperature=temperature)
        
        # Save the melody as a MIDI file
        midi_file = os.path.join(OUTPUT_DIR, f'generated_melody_{i+1}.mid')
        save_midi(sequence, midi_file)
        print(f"Saved MIDI file: {midi_file}")
        
        # Convert the MIDI file to an audio file
        try:
            audio_file = os.path.join(OUTPUT_DIR, f'generated_melody_{i+1}.wav')
            midi_to_audio(midi_file, audio_file)
            print(f"Saved audio file: {audio_file}")
        except Exception as e:
            print(f"Error converting MIDI to audio: {e}")

if __name__ == "__main__":
    print("AI Music Generation Script")
    print("=========================")
    print("Downloading models (if this is your first run)...")
    
    # Download model if it doesn't exist
    if not os.path.exists(BUNDLE_PATH):
        print("Downloading melody_rnn model... This might take a while.")
        import urllib.request
        url = "https://storage.googleapis.com/magentadata/models/melody_rnn/lookback_rnn.mag"
        urllib.request.urlretrieve(url, BUNDLE_PATH)
        print("Model downloaded successfully!")
    
    # Generate music
    num_sequences = int(input("How many music sequences would you like to generate? "))
    temperature = float(input("Enter temperature (0.1-2.0, lower is more conservative): "))
    
    generate_and_save_music(num_sequences, temperature)
    
    print("\nGeneration complete! The files are saved in the", OUTPUT_DIR, "directory.")
    print("You can play the .wav files to hear your AI-generated music!")

ModuleNotFoundError: No module named 'magenta'

## Requirements and Setup

To use this code, you'll need to install several packages:

```
pip install magenta midi2audio tensorflow numpy
```

You'll also need to install FluidSynth for MIDI to audio conversion:
- **Windows**: Download from https://www.fluidsynth.org/ or use `choco install fluidsynth`
- **macOS**: `brew install fluid-synth`
- **Linux**: `apt-get install fluidsynth`

## How the Code Works

1. The script uses Magenta's MelodyRNN model to generate music
2. It downloads a pre-trained model called "lookback_rnn" (one of Magenta's melody generation models)
3. The music is first generated as a MIDI file
4. Then it's converted to an audio WAV file that you can play

## Customization Options

You can adjust these parameters when running the script:
- **Number of sequences**: How many different melodies to generate
- **Temperature**: Controls randomness (lower values = more predictable, higher values = more experimental)

Would you like me to explain any part of this code in more detail? Or would you like guidance on more advanced AI music generation approaches?